In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
import pickle
import numpy as np

import crflux.models as pm

from MCEq.data import InteractionCrossSections
from MCEq.core import MCEqRun
import  mceq_underground_helpers_oneregion_threshold as helper
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [2]:
def sl_depth(d,angle):
    return d/np.cos(np.deg2rad(angle))

In [3]:
plot_dir = '/hetghome/khymon/Plots/cs-analysis/'


X = helper.slant_depths[2:-1:4]

In [4]:
cs_p_values = [1.0,1.01] #[0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5]  # List of cross-section values: pion-air
cs_k_values = [1.00]
cs_pr_values = [1.00]
ptype_values = [2212] 
season_values = ["jan", "apr", "jul"]  #  seasons
e0_values = [2.05,2.15,2.25,2.35,2.45,2.55,2.65,2.75,2.85,2.95,3.05,3.15,3.25,3.35,3.45,3.55,3.65,3.75,3.85,3.95,4.05,4.15,4.25,4.35,4.45,4.55,4.65,4.75,4.85]
e1_values = [2.25,2.35,2.45,2.55,2.65,2.75,2.85,2.95,3.05,3.15,3.25,3.35,3.45,3.55,3.65,3.75,3.85,3.95,4.05,4.15,4.25,4.35,4.45,4.55,4.65,4.75,4.85,4.95,5.05]
e_center = e0_values + np.ones(len(e0_values))*0.1
pairwise = True


helper.initialize_flux_dicts(
        ptype_values, cs_p_values, cs_k_values, cs_pr_values, e0_values, e1_values if 'e1_values' in locals() else [None], pairwise=pairwise
) 

helper.some_function_that_uses_angles()
angles = helper.angles

2.05 2.25
helper script 2.05 2.25 1.01 1.0 1.0
2.15 2.35
helper script 2.15 2.35 1.01 1.0 1.0
2.25 2.45
helper script 2.25 2.45 1.01 1.0 1.0
2.35 2.55
helper script 2.35 2.55 1.01 1.0 1.0
2.45 2.65
helper script 2.45 2.65 1.01 1.0 1.0
2.55 2.75
helper script 2.55 2.75 1.01 1.0 1.0
2.65 2.85
helper script 2.65 2.85 1.01 1.0 1.0
2.75 2.95
helper script 2.75 2.95 1.01 1.0 1.0
2.85 3.05
helper script 2.85 3.05 1.01 1.0 1.0
2.95 3.15
helper script 2.95 3.15 1.01 1.0 1.0
3.05 3.25
helper script 3.05 3.25 1.01 1.0 1.0
3.15 3.35
helper script 3.15 3.35 1.01 1.0 1.0
3.25 3.45
helper script 3.25 3.45 1.01 1.0 1.0
3.35 3.55
helper script 3.35 3.55 1.01 1.0 1.0
3.45 3.65
helper script 3.45 3.65 1.01 1.0 1.0
3.55 3.75
helper script 3.55 3.75 1.01 1.0 1.0
3.65 3.85
helper script 3.65 3.85 1.01 1.0 1.0
3.75 3.95
helper script 3.75 3.95 1.01 1.0 1.0
3.85 4.05
helper script 3.85 4.05 1.01 1.0 1.0
3.95 4.15
helper script 3.95 4.15 1.01 1.0 1.0
4.05 4.25
helper script 4.05 4.25 1.01 1.0 1.0
4.15 4.35
hel

In [5]:
cs_p = 1.01
cs_k = 1.0
cs_pr = 1.0
angle = 0.0
season = "apr"

In [6]:
def get_log_y(X, angle,cs_p,cs_k,cs_pr,e0,e1,season,threshold):
    log_y = np.log10(1e4*helper.mult_dist(X,
                            angle,
                            pm.GlobalSplineFitBeta(),"yields_" + season,
                            2212,
                            cs_p, cs_k,cs_pr,e0,e1,
                            norm=False,threshold=threshold#))[0]
                        ) / helper.rates(X, angle, season, 2212, cs_p, cs_k,cs_pr, e0,e1,threshold))

    return log_y

In [7]:
def get_derivative(x_log, log_y,m_low,m_high):

    deriv_low = (log_y[m_low] - log_y[1])/(np.log10(x_log[m_low])-np.log10(x_log[1])) 
    deriv_high = (log_y[99] - log_y[m_high])/(np.log10(x_log[99])-np.log10(x_log[m_high])) 

    return np.array([deriv_low, deriv_high])

In [8]:
x_log = np.log10(helper.n_mu_vec)

In [ ]:
# add integration limits as argument

sl = sl_depth(1.5,angles)

deriv_mod_sl_500 =np.ndarray(shape=(len(sl),len(e0_values),49,49,2))
deriv_def_sl_500 = np.ndarray(shape=(len(sl),49,49,2))

multi_low = np.arange(1,49)
multi_high = np.arange(50,99)

for s in range(len(sl)):    
    for l in range(len(multi_low)):
        for h in range(len(multi_high)):

            y_log_def_sl_500 = get_log_y(sl[s], angle, 1.0, 1.0 ,1.0, 2.05,"inf",season,threshold=500)
            deriv_def_sl_500[s][l][h] = get_derivative(x_log, y_log_def_sl_500,multi_low[l],multi_high[h])

            for en in range(len(e0_values)):
                logy_mod_500 = get_log_y(sl[s], angle, cs_p, cs_k ,cs_pr, e0_values[en],e1_values[en],season,threshold=500)
                deriv_mod_sl_500[s][en][l][h] = get_derivative(x_log, logy_mod_500,multi_low[l],multi_high[h])


/tmp/ipykernel_1122076/3540314797.py:3: RuntimeWarning: invalid value encountered in scalar divide
  deriv_low = (log_y[m_low] - log_y[1])/(np.log10(x_log[m_low])-np.log10(x_log[1]))


In [ ]:
# get sensitivity

def_ratio_sl_500 = np.ones(len(sl))* (deriv_def_sl_500[:,:,:,0]/deriv_def_sl_500[:,:,:,1])


delta_P_500 = (deriv_mod_sl_500[:,:,:,:0]/deriv_mod_sl_500[:,:,:,:,1])-def_ratio_sl_500[:,np.newaxis]
dP_dsig_500 = delta_P_500/0.01